In [1]:
import requests
import arcpy
import pandas as pd

In [2]:
directory = r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb"

In [3]:
landcover_raw= "C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\MN_NLCD_2019\MN_NLCD_2019_Land_Cover.tif"

# Check SRS
def check_sr(in_raster, in_sr):
    sr = arcpy.Describe(in_raster).spatialReference
    in_sr_code = arcpy.SpatialReference(in_sr)
    
    if sr.factoryCode == in_sr_code.factoryCode:
        print("Coordinate Systems Match")
        
    else:
        print("Coordinate Systems Do Not Match")
        print(f"Actual Coordinate System is: {sr.factoryCode}")

check_sr(landcover_raw, 26915)

Coordinate Systems Match


In [4]:
# Check Bounding Box
def check_box(in_raster, ymin, ymax, xmin, xmax):
    
    left = float(arcpy.management.GetRasterProperties(in_raster, "LEFT").getOutput(0))
    right = float(arcpy.management.GetRasterProperties(in_raster,"RIGHT").getOutput(0))
    top = float(arcpy.management.GetRasterProperties(in_raster, "TOP").getOutput(0))
    bottom = float(arcpy.management.GetRasterProperties(in_raster,"BOTTOM").getOutput(0))
    
    print(left, right , top , bottom)
    
    if left < xmin or right > xmax or top > ymax or bottom < ymin:
        print("Raster is not Within Bounding Box")
    
    else: 
        print("Raster is Within Bounding Box")
        
check_box(landcover_raw, 4770648.492016879, 5490732.38301064, 133186.3828544069, 789594.190423549)

-112390.0 801260.0 5566390.0 4600900.0
Raster is not Within Bounding Box


In [5]:
arcpy.management.Clip(
    in_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\Lab2.gdb\landcover",
    rectangle="190687.908114248 4816425.97098429 758772.057946726 5471006.46677986",
    out_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\landcover_Clip",
    in_template_dataset=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\Lab2\Lab2.gdb\state_of_minnesota",
    nodata_value="255",
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\bmsb\\bmsb.gdb\\landcover_Clip'>

In [6]:
arcpy.ddd.Reclassify(
    in_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\landcover_Clip",
    reclass_field="VALUE",
    remap="11 5;21 4;22 4;23 4;24 4;31 4;41 2;42 2;43 2;52 2;71 2;81 3;82 3;90 1;95 1",
    out_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\land_reclass5",
    missing_values="DATA"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\bmsb\\bmsb.gdb\\land_reclass5'>

In [7]:
arcpy.conversion.RasterToGeodatabase(
    Input_Rasters=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\land_reclass5",
    Output_Geodatabase=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb",
    Configuration_Keyword=""
)

<Result ''>

In [8]:
arcpy.management.Resample(
    in_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\land_reclass5",
    out_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\land_reclass5_Resample1k",
    cell_size="1000 1000",
    resampling_type="NEAREST"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\bmsb\\bmsb.gdb\\land_reclass5_Resample1k'>

In [9]:
arcpy.conversion.RasterToPoint(
    in_raster=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\land_reclass5_Resample1k",
    out_point_features=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\lc_point",
    raster_field="Value"
)


<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\bmsb\\bmsb.gdb\\lc_point'>

In [10]:
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=r"C:\Mac\Home\Documents\ArcGIS\Projects\misc\ctus_with_id\ctus_with_id.shp",
    Output_Geodatabase=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\bmsb\\bmsb.gdb'>

In [11]:
arcpy.analysis.SummarizeWithin(
    in_polygons="ctus_with_id",
    in_sum_features="lc_point",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\ctus_nlcd_SummarizeWithin",
    keep_all_polygons="KEEP_ALL",
    sum_fields="grid_code Mean",
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="SQUAREKILOMETERS",
    group_field="grid_code",
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="ADD_PERCENT",
    out_group_table=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5572\bmsb\bmsb.gdb\ncld_ctu_summary"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5572\\bmsb\\bmsb.gdb\\ctus_nlcd_SummarizeWithin'>

In [12]:
arcpy.management.AddField(
    in_table="ncld_ctu_summary",
    field_name="lc_class",
    field_type="TEXT",
    field_precision=None,
    field_scale=None,
    field_length=255,
    field_alias="",
    field_is_nullable="NULLABLE",
    field_is_required="NON_REQUIRED",
    field_domain=""
)

<Result 'ncld_ctu_summary'>

In [13]:
standalone_table = "ncld_ctu_summary" 


land_cover_dict = {
    1: "Wetlands",
    2: "Forest",
    3: "Agriculture",
    4: "Developed",
    5: "Water"
}

with arcpy.da.UpdateCursor(standalone_table, ["grid_code", "lc_class"]) as cursor:
    for row in cursor:
        grid_code = row[0]
        if grid_code in land_cover_dict:
            row[1] = land_cover_dict[grid_code]
            cursor.updateRow(row)